In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 
from tensorflow.keras.utils import Sequence
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import math
import os 
import cv2 as cv

In [2]:
#bucket_name = 'price-match-guarantee'
#create_bucket(bucket_name)

In [3]:
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [4]:
EPOCHS = 16

In [5]:
Batch_size = 8 * strategy.num_replicas_in_sync

In [6]:
test = pd.read_csv("../input/shopee-product-matching/train.csv")

In [7]:
test.head()

,posting_id,image,image_phash,title,label_group
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069


In [8]:
N_classes = test.label_group.nunique()
path_image = "../input/shopee-product-matching/train_images"

In [9]:
lb = LabelEncoder()
test["encoded_label_group"] = lb.fit_transform(test["label_group"].values)

In [10]:
img_size = 512

In [11]:
class Datagenerator(Sequence):
     def __init__(self,batch_size = Batch_size ,path=path_image,listID = None,\
                  shuffle=True,img_size=img_size):
         self.batch_size = batch_size 
         self.path = path 
         self.listID = listID
         self.shuffle = shuffle
         self.img_size = img_size
         self.on_epoch_end()
         
     def __len__(self):
         l = len(self.listID) // self.batch_size
         l += int((len(self.listID) % self.batch_size) !=0)
         return l 
     def on_epoch_end(self):
         self.indexes = np.arange(len(self.listID))
         if self.shuffle :
            np.random.shuffle(self.indexes)
     def __getitem__(self,index):
         indices = self.indexes[index * self.batch_size:(index+1) * self.batch_size]
         list_ID_temp = [self.listID[k] for k in indices]
         X,Y = self.__datageneration(list_ID_temp)
         X = X,Y
         Y = Y
         return X,Y
     def __datageneration(self,list_id):
         image = np.zeros((len(list_id),self.img_size,self.img_size,3),dtype=np.float)
         df = test.iloc[list_id]
         y = np.zeros((len(list_id)),dtype=int)
         for i , (j ,row) in enumerate(df.iterrows()):
             img = row.image
             img = os.path.join(self.path,img)
             img = cv.imread(img)
             img = cv.resize(img,(self.img_size,self.img_size))
             image[i,] = img 
             y[i] = row.encoded_label_group
         return image,y
         

In [12]:
tr_ind,ts_ind,tr_lab,ts_lab = train_test_split(test.index,test.encoded_label_group.values,\
                                               test_size=0.33,stratify=test.encoded_label_group.values)

In [13]:
train_data = Datagenerator(listID=tr_ind)
val_data = Datagenerator(listID=ts_ind)

In [14]:
class ArcMarginProduct(tf.keras.layers.Layer):
    '''
    Implements large margin arc distance.

    Reference:
        https://arxiv.org/pdf/1801.07698.pdf
        https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/
            blob/master/src/modeling/metric_learning.py
    '''
    def __init__(self, n_classes, s=30, m=0.50, easy_margin=False,
                 ls_eps=0.0, **kwargs):

        super(ArcMarginProduct, self).__init__(**kwargs)

        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.ls_eps = ls_eps
        self.easy_margin = easy_margin
        self.cos_m = tf.math.cos(m)
        self.sin_m = tf.math.sin(m)
        self.th = tf.math.cos(math.pi - m)
        self.mm = tf.math.sin(math.pi - m) * m

    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'ls_eps': self.ls_eps,
            'easy_margin': self.easy_margin,
        })
        return config

    def build(self, input_shape):
        super(ArcMarginProduct, self).build(input_shape[0])

        self.W = self.add_weight(
            name='W',
            shape=(int(input_shape[0][-1]), self.n_classes),
            initializer='glorot_uniform',
            dtype='float32',
            trainable=True,
            regularizer=None)

    def call(self, inputs):
        X, y = inputs
        y = tf.cast(y, dtype=tf.int32)
        cosine = tf.matmul(
            tf.math.l2_normalize(X, axis=1),
            tf.math.l2_normalize(self.W, axis=0)
        )
        sine = tf.math.sqrt(1.0 - tf.math.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = tf.where(cosine > 0, phi, cosine)
        else:
            phi = tf.where(cosine > self.th, phi, cosine - self.mm)
        one_hot = tf.cast(
            tf.one_hot(y, depth=self.n_classes),
            dtype=cosine.dtype
        )
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.n_classes

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output

In [15]:
def get_lr_callback():
    lr_start   = 0.000001
    lr_max     = 0.000005 * Batch_size
    lr_min     = 0.000001
    lr_ramp_ep = 5
    lr_sus_ep  = 0
    lr_decay   = 0.8
   
    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start   
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max    
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min    
        return lr

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)
    return lr_callback


In [16]:
def _regularizer(weights_decay=5e-4):
    return tf.keras.regularizers.l2(weights_decay)

In [17]:

def build_model(N_classes,training=False,emb_size = 256,w_decay=5e-4):
    with strategy.scope():
        input_image = tf.keras.layers.Input(shape=(img_size,img_size,3),name="input_image")
        base_model = tf.keras.applications.EfficientNetB3(weights="imagenet",include_top=False,\
                                                 input_shape=(img_size,img_size,3),pooling="avg")
        base_model.trainable = True
        x = base_model(input_image)
        #x = tf.keras.layers.GlobalAveragePooling2D()(x)
        emb = tf.keras.layers.BatchNormalization()(x)
        if training :
            margin = ArcMarginProduct(
            n_classes = N_classes, 
            s = 30, 
            m = 0.5, 
            name='head/arc_margin', 
            dtype='float32'
            )
            y = tf.keras.layers.Input(shape=(),name="labels")
            logits = margin([emb,y])
            logits = tf.keras.layers.Softmax()(logits)
            return tf.keras.models.Model((input_image,y),logits)
        else :
            return tf.keras.models.Model(input_image,emb)

In [18]:
md = build_model(N_classes,training=True)

43941888/43941136 [==============================] - 0s 0us/step


In [19]:
md.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
efficientnetb3 (Functional)     (None, 1536)         10783535    input_image[0][0]                
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 1536)         6144        efficientnetb3[0][0]             
__________________________________________________________________________________________________
labels (InputLayer)             [(None,)]            0                                            
______________________________________________________________________________________________

In [20]:
md.compile(optimizer=tf.keras.optimizers.Adam(1e-5),loss=[tf.keras.losses.SparseCategoricalCrossentropy()],metrics="accuracy")

In [21]:
#checkpoints = tf.keras.callbacks.ModelCheckpoint(f"effnet.b3",monitor="val_loss",\
                                                 #save_best_only=True,save_weights_only=True,\
                                                 #mode="min",verbose=1)

In [22]:
steps_per_epoch = len(tr_ind) // Batch_size


val_steps_per_epoch = len(ts_ind) // Batch_size
md.fit(train_data,epochs= EPOCHS,validation_data=val_data,\
      callbacks=[get_lr_callback()])

Epoch 1/16

Epoch 00001: LearningRateScheduler reducing learning rate to 1e-06.
2869/2869 [==============================] - 1874s 647ms/step - loss: 23.9441 - accuracy: 0.0000e+00 - val_loss: 23.8844 - val_accuracy: 0.0000e+00
Epoch 2/16

Epoch 00002: LearningRateScheduler reducing learning rate to 8.800000000000002e-06.
2869/2869 [==============================] - 1817s 633ms/step - loss: 23.7788 - accuracy: 0.0000e+00 - val_loss: 23.3892 - val_accuracy: 0.0000e+00
Epoch 3/16

Epoch 00003: LearningRateScheduler reducing learning rate to 1.6600000000000004e-05.
2869/2869 [==============================] - 1795s 625ms/step - loss: 23.0399 - accuracy: 0.0000e+00 - val_loss: 22.4266 - val_accuracy: 0.0000e+00
Epoch 4/16

Epoch 00004: LearningRateScheduler reducing learning rate to 2.4400000000000007e-05.
2869/2869 [==============================] - 1812s 631ms/step - loss: 21.7186 - accuracy: 0.0000e+00 - val_loss: 21.0593 - val_accuracy: 0.0000e+00
Epoch 5/16

Epoch 00005: LearningRateS

In [23]:
md.save("model")

In [24]:
def get_layer_index(model, layer_name, not_found=None):
    """get model's layer index by layer's name"""
    for i, layer in enumerate(model.layers):
        if layer.name == layer_name:
            return i
    return not_found

In [25]:
md.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
efficientnetb3 (Functional)     (None, 1536)         10783535    input_image[0][0]                
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 1536)         6144        efficientnetb3[0][0]             
__________________________________________________________________________________________________
labels (InputLayer)             [(None,)]            0                                            
______________________________________________________________________________________________

In [26]:
model = tf.keras.models.Model(inputs=md.layers[-5].input, outputs=md.layers[-5].output)

In [27]:
model.save("embadding_arcface")